# 기본 설정

구글 드라이브 마운트

패키지 설치

In [1]:
#from google.colab import drive
#drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [1]:
import torch

In [ ]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

torch.backends.mps.is_built() 
torch.backends.mps.is_available()

tensor([1.], device='mps:0')


/opt/miniconda3/envs/noise_control/lib/python3.9/site-packages/torch/_tensor_str.py:137: UserWarning: MPS: nonzero op is supported natively starting from macOS 13.0. Falling back on CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Indexing.mm:283.)
  nonzero_finite_vals = torch.masked_select(


True

In [4]:
!pip install transformers
!pip install datasets
!pip install accelerate

In [5]:
import pandas as pd
import numpy as np
from glob import glob
import os
import tqdm
from transformers import (
    AutoTokenizer,
    AutoModel,
    DataCollatorWithPadding
)

from datasets import Dataset, Value
import torch
import torch.nn as nn
from typing import List, Optional, Tuple, Union
import datetime
import re

/opt/miniconda3/envs/noise_control/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
#@title 마이너 패키지 로딩
%matplotlib inline
from datetime import datetime
import matplotlib.pyplot as plt
#plt.style.use('seaborn-darkgrid')

In [7]:
#cd /gdrive/MyDrive/Lectures/2025/RecSys/content-based

# 데이터 로드

`../raw/ml-100k`
- u1.base: 학습 (80K)
- u1.test: 검증 (20K)

In [22]:
#@title get_user_item_map
def get_user_item_map(X):
  """Function to generate a ratings matrx and mappings for
  the user and item ids to the row and column indices

  Parameters
  ----------
  X : pandas.DataFrame, shape=(n_ratings,>=3)
      First 3 columns must be in order of user, item, rating.

  Returns
  -------
  user_map : pandas Series, shape=(n_users,)
      Mapping from the original user id to an integer in the range [0,n_users)
  item_map : pandas Series, shape=(n_items,)
      Mapping from the original item id to an integer in the range [0,n_items)
  """
  user_col, item_col, rating_col = X.columns[:3]
  rating = X[rating_col]
  user_map = pd.Series(
      index=np.unique(X[user_col]),
      data=np.arange(X[user_col].nunique()),
      name='user_map',
  )
  item_map = pd.Series(
      index=np.unique(X[item_col]),
      data=np.arange(X[item_col].nunique()),
      name='columns_map',
  )

  return user_map, item_map

In [9]:
item_plot_df = pd.read_csv('movie_plots_80_missings.csv', index_col=0)

In [10]:
def load_data(file_path):
  ratings_df = pd.read_csv(file_path, sep='\t', header=None,
                          names=['userId', 'movieId', 'rating', 'timestamp'])
  ratings_df['timestamp'] = ratings_df['timestamp'].apply(datetime.fromtimestamp)
  ratings_df = ratings_df.sort_values('timestamp')
  return ratings_df

In [11]:
train_df = load_data('raw/ml-100k/u1.base').reset_index(drop=True)
val_df = load_data('raw/ml-100k/u1.test').reset_index(drop=True)

In [12]:
user_map, item_map = get_user_item_map(pd.concat((train_df, val_df), axis=0))

In [13]:
train_df['plots'] = train_df.movieId.map(item_plot_df['plot'])
val_df['plots'] = val_df.movieId.map(item_plot_df['plot'])

# 모델링

컨텐츠 모델링
- 플롯(plot) 활용: Distilbert (https://huggingface.co/docs/transformers/model_doc/distilbert#distilbert)

유저 모델링
- 단순 임베딩

결합
- 연결: concatenation

예측
- Linear regression

아이템 플롯 데이터
- movie_plots_80_missings.csv

참고
- ~~https://ratsgo.github.io/nlpbook/docs/lm~~
- https://huggingface.co/docs/transformers/index


In [15]:
pretrained_model_tag = "distilbert-base-uncased"

## 데이터셋

In [16]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_tag)

In [17]:
def preprocess_function(examples):
    return tokenizer(examples["plots"], truncation=True, max_length=256)

데이터셋 변환
- `Dataset.map(fn)`

In [18]:
def make_dataset(df):
    ds = Dataset.from_pandas(pd.concat([df.userId.map(user_map),
                              df.movieId.map(item_map),
                              df.rating,
                              df.plots],
                             axis=1)
                            )
    ds = ds.map(preprocess_function, batched=True)
    ds = ds.rename_column("userId", "user_id")
    ds = ds.rename_column("movieId", "movie_id")
    ds = ds.rename_column("rating", "label")
    ds = ds.cast_column('label', Value("float"))
    return ds.remove_columns(['plots', 'movie_id'])

토크나이징

In [19]:
train_ds = make_dataset(train_df)
validation_ds = make_dataset(val_df)

Casting the dataset: 100%|██████████| 20000/20000 [00:00<00:00, 776162.40 examples/s]


포맷 변경
- `torch` 포맷으로...

In [20]:
train_ds = train_ds.with_format('torch').shuffle(
        seed=0
    )
validation_ds = validation_ds.with_format('torch')


In [23]:
## 학습데이터 확인 ##

print(validation_ds)


print(validation_ds[0]['input_ids'].shape)
#input 3개 출력
for i in range(3):
    print(f"Sample {i} input_ids:", validation_ds[i]['input_ids'])


#label 3개 출력
labels = torch.tensor([example['label'] for example in validation_ds])
#print(labels.shape)  # 예: torch.Size([25000])
print(labels[:3])   # 앞 3개 출력

Dataset({
    features: ['user_id', 'label', 'input_ids', 'attention_mask'],
    num_rows: 20000
})
torch.Size([256])
Sample 0 input_ids: tensor([  101,  2043,  1037,  2450,  1005,  1055,  2146,  1011,  2051,  2767,
         7657,  2002,  1005,  1055,  5117,  1010,  2016, 10919,  2016,  7459,
         2032,  2841,  1998,  4520,  2041,  2000,  2131,  2032,  1010,  2007,
         2069,  2420,  2077,  1996,  5030,  1012,  1037,  2450,  2040,  1010,
         2011,  1037,  4872,  2081,  2086,  3041,  1010,  2003,  4011,  2000,
         5914,  2014,  2190,  2767,  1999,  2093,  3134,  1010,  2130,  2295,
         2016,  2987,  1005,  1056,  2215,  2000,  1012,  2043,  2016,  4858,
         2041,  2008,  2002,  1005,  1055, 13516,  2619,  2842,  1010,  2016,
         4150,  9981,  1998,  5363,  2000,  3338,  2125,  1996,  5030,  1012,
         1024,  1024,  2728,  1047, 24175, 19779,  5488,  1026,  2962, 18927,
         6508,  9905,  1030,  7570, 21397,  1012,  4012,  1028,  2745,  1051,
    

## DataCollatorWithPadding

모델에 전달할 최종 입력과 출력 가공
- 만약 입력이 B개의 미니 배치로 구성될 때
- 각 토큰 시퀀스인 `input_ids`의 길이가 다르다면
- 끝을 패딩하여, 미니 배치 내의 모든 시퀀스의 길이가 동일하게 해주는 역할
- (필요시 출력도 변경...)

In [24]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### DataCollator 테스트

In [25]:
inputs = train_ds[:16]

inputs_len = [len(x) for x in inputs['input_ids']]

# 입력 시퀀스의 길이 조사
print(inputs_len)

# 문장들을 배치로 묶고 패딩합니다.
batch = data_collator(inputs)

# 결과를 출력합니다.
#print(batch['input_ids'][15])
print(batch['input_ids'][2])

[210, 256, 256, 256, 210, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 110]
tensor([  101,  4079, 27850,  1006,  1996,  3002,  1007,  1010,  2003,  1037,
        12383,  2005, 10887,  1010,  3005,  6745,  3105,  2000,  8954,  1996,
         3595,  2832,  2005,  3147, 10077,  8509,  2032,  2012, 10238,  2007,
         1037, 17328,  6260,  2006,  2327, 14353,  1996,  2845,  2231,  1010,
         2004,  2092,  2004,  1996,  2450,  2040,  4324,  2049,  3595,  1012,
         4079, 27850,  2038,  2053,  2613,  2155,  1010,  2053,  2613,  2188,
         1010,  1998,  4079, 27850,  3475,  1005,  1056,  2130,  2010,  2613,
         2171,  1012,  2664,  1010,  4079, 27850,  1010,  2036,  2124,  2004,
         1996,  3002,  2005,  2010,  2224,  1997,  4526,  6270, 15702,  2478,
         1996,  3415,  1997,  3234,  6586,  1010,  2003,  2028,  1997,  1996,
         2088,  1005,  1055,  2087,  3144, 15862,  1012, 13554,  1010,  2139,
        11735, 11215,  1010,  1998,  1037,  3040,  1997, 1424

# 모델 정의

모델 설정과 모델 클래스 생성
- DistilBertForRegressionConfig
- DistilBertForRegression

In [26]:
from transformers import DistilBertConfig
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.models.distilbert.modeling_distilbert import DistilBertModel
from transformers.models.distilbert.modeling_distilbert import DistilBertPreTrainedModel

# DistilBertConfig를 상속받아 새로운 설정 클래스 정의
class DistilBertForRegressionConfig(DistilBertConfig):
    def __init__(self, user_num, factor_num, regressor_dropout, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.user_num = user_num
        self.factor_num = factor_num
        self.regressor_dropout = regressor_dropout

# DistilBertModel 바디에 Regression 헤드를 추가한 클래스 정의
class DistilBertForRegression(DistilBertPreTrainedModel):
    config_class = DistilBertForRegressionConfig

    def __init__(self, config):
        super().__init__(config)

        # 유저 임베딩 레이어
        self.embed_user = nn.Embedding(config.user_num, config.factor_num)
        # 모델 바디
        self.distilbert = DistilBertModel(config)
        # 예측 헤드
        self.regressor_dropout = nn.Dropout(config.regressor_dropout)
        self.regressor = nn.Linear(config.hidden_size + config.factor_num, config.num_labels)

        # 가중치를 로드하고 초기화
        self.init_weights()

    def forward(self, user_id=None, input_ids=None, attention_mask=None,
                labels=None, **kwargs):
        
        kwargs.pop('num_items_in_batch', None)
        # 유저 임베딩 생성
        embed_user_output = self.embed_user(user_id)

        # distilbert 출력 생성하고, 맨 앞 토큰의 히든 임베딩을 조회
        distilbert_output = self.distilbert(input_ids=input_ids, attention_mask=attention_mask,
                               **kwargs)
        pooled_output = distilbert_output.last_hidden_state[:, 0]

        # 유저 임베딩과 히든 임베딩을 붙임
        concat = torch.cat((embed_user_output, pooled_output), -1) # (B, hidden_size+factor_num)

        # dropout 통과 및 회귀 결과 생성
        sequence_output = self.regressor_dropout(concat)
        logits = self.regressor(sequence_output).view(-1) # (B, 1) -> (B, )

        # 손실계산
        loss = None
        if labels is not None:
            loss_fct = nn.MSELoss()
            loss = loss_fct(logits, labels)

        return SequenceClassifierOutput(loss=loss, logits=logits)

## 설정 초기화

In [27]:
from transformers import AutoConfig

distil_config = AutoConfig.from_pretrained(pretrained_model_tag,
                                           num_labels=1
                                          )
distil_config.user_num=len(user_map)
distil_config.factor_num=32
distil_config.regressor_dropout=0.2

## 모델 로딩

In [28]:
distil_model = DistilBertForRegression.from_pretrained(pretrained_model_tag, config=distil_config)

Some weights of DistilBertForRegression were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['embed_user.weight', 'regressor.bias', 'regressor.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 모델 아키텍쳐 관찰

In [29]:
distil_model

DistilBertForRegression(
  (embed_user): Embedding(943, 32)
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(

In [30]:
# distil_model(**batch)

## 훈련 변수 설정

In [31]:
from transformers import TrainingArguments

num_epochs = 3
# 코랩에서 GPU 메모리 부족 에러가 나는 경우 batch_size를 16,32 등으로 줄여보자
batch_size = 64
logging_steps = len(train_ds) // batch_size
model_name = f"distilbert-fine-tuned"
training_args = TrainingArguments(
    output_dir=model_name, log_level="error", num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size, eval_strategy="epoch",
    save_steps=1e6, weight_decay=0.01, disable_tqdm=False,
    logging_steps=logging_steps, push_to_hub=False)

## 메트릭 정의 (MAE)

In [32]:
from sklearn.metrics import mean_absolute_error

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.squeeze()  # if necessary
    return {'mae': mean_absolute_error(labels, predictions)}

## 트레이너 생성

In [33]:
from transformers import Trainer

trainer = Trainer(model=distil_model, args=training_args,
                  data_collator=data_collator, compute_metrics=compute_metrics,
                  train_dataset=train_ds,
                  eval_dataset=validation_ds,
                  tokenizer=tokenizer)

/var/folders/4n/97qjc42s3934wcg3h06zcych0000gn/T/ipykernel_41188/3681518013.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=distil_model, args=training_args,


In [34]:
distil_model.config.use_cache = False

# 훈련 실시

수행시간
- Colab T4: 1시간30분
- RTX 4090: 8분 30초
- Titan RTX: 27분 10초
- Macpro M1_max: 1시간23분

In [35]:
train_result = trainer.train()

Epoch,Training Loss,Validation Loss,Mae
1,1.141600,1.087039,0.824280
2,1.040900,1.092121,0.822595
3,1.010300,1.049528,0.814847


## 훈련 결과 저장

In [36]:
trainer.save_model()

## 예측

In [37]:
predictions = trainer.predict(validation_ds)


In [38]:
sorted(zip(predictions.predictions, validation_ds['label'].numpy()), key=lambda x: x[0])[:100]


[(1.9188912, 1.0),
 (1.9347687, 1.0),
 (1.987005, 1.0),
 (1.9890965, 1.0),
 (1.9941037, 1.0),
 (2.004974, 1.0),
 (2.0071006, 1.0),
 (2.0083272, 1.0),
 (2.0106902, 2.0),
 (2.0115013, 1.0),
 (2.012013, 1.0),
 (2.0151231, 1.0),
 (2.0211954, 1.0),
 (2.0243669, 3.0),
 (2.0268574, 1.0),
 (2.0327454, 1.0),
 (2.0329943, 2.0),
 (2.0572102, 1.0),
 (2.0613518, 3.0),
 (2.0643582, 4.0),
 (2.064654, 1.0),
 (2.0664291, 3.0),
 (2.0673385, 1.0),
 (2.0702796, 1.0),
 (2.0733643, 3.0),
 (2.0734317, 1.0),
 (2.0785325, 1.0),
 (2.0823674, 1.0),
 (2.0909634, 3.0),
 (2.0969539, 3.0),
 (2.098423, 1.0),
 (2.0985918, 1.0),
 (2.099485, 2.0),
 (2.0996778, 1.0),
 (2.1062465, 1.0),
 (2.106304, 2.0),
 (2.1069317, 1.0),
 (2.1094403, 1.0),
 (2.1103594, 3.0),
 (2.1120958, 3.0),
 (2.1164465, 4.0),
 (2.1176982, 2.0),
 (2.1179123, 2.0),
 (2.1189203, 4.0),
 (2.1211483, 4.0),
 (2.121932, 1.0),
 (2.1224303, 3.0),
 (2.1239653, 1.0),
 (2.1255815, 3.0),
 (2.128141, 4.0),
 (2.1284375, 3.0),
 (2.129563, 1.0),
 (2.1303864, 1.0),
 (2

In [39]:
sorted(zip(predictions.predictions, validation_ds['label'].numpy()), key=lambda x: x[0])[-100:]

[(4.5639343, 4.0),
 (4.5646324, 5.0),
 (4.5648265, 5.0),
 (4.565353, 5.0),
 (4.5655136, 5.0),
 (4.5655694, 4.0),
 (4.5681753, 5.0),
 (4.5684333, 5.0),
 (4.569118, 5.0),
 (4.569959, 5.0),
 (4.5704103, 4.0),
 (4.5704885, 5.0),
 (4.571491, 1.0),
 (4.571973, 4.0),
 (4.5721765, 5.0),
 (4.5731106, 4.0),
 (4.574251, 5.0),
 (4.574787, 4.0),
 (4.5751376, 4.0),
 (4.5755534, 5.0),
 (4.5758653, 5.0),
 (4.5761213, 5.0),
 (4.5766215, 5.0),
 (4.577599, 5.0),
 (4.5783644, 4.0),
 (4.5786834, 5.0),
 (4.5800796, 5.0),
 (4.5819006, 3.0),
 (4.5819335, 4.0),
 (4.5822997, 1.0),
 (4.5830164, 5.0),
 (4.583258, 4.0),
 (4.584028, 5.0),
 (4.585916, 5.0),
 (4.586161, 4.0),
 (4.5863304, 4.0),
 (4.587627, 4.0),
 (4.5881886, 3.0),
 (4.5883865, 4.0),
 (4.58868, 5.0),
 (4.5888815, 5.0),
 (4.5900717, 5.0),
 (4.590628, 5.0),
 (4.591888, 3.0),
 (4.592698, 5.0),
 (4.594973, 3.0),
 (4.597542, 4.0),
 (4.598497, 5.0),
 (4.601621, 5.0),
 (4.6031046, 5.0),
 (4.605293, 5.0),
 (4.6053843, 3.0),
 (4.6059957, 5.0),
 (4.6064844, 5.0

In [40]:
preds = predictions.predictions.flatten()  # 예측값 (1D로 변형)
labels = validation_ds['label'].numpy()
mae = mean_absolute_error(labels, preds)
print(f"MAE: {mae:.4f}")

MAE: 0.8148
